In [1]:
 !pip install datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing

In [34]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00


In [22]:
from datasets import load_dataset
from transformers import AutoTokenizer

def load_and_tokenize(model_name="bert-base-uncased"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    dataset = load_dataset("glue", "sst2")

    def tokenize(batch):
        return tokenizer(
            batch["sentence"],
            padding="max_length",   # ensures fixed-length tensors
            truncation=True,
            max_length=128,         # you can tweak this
        )

    tokenized = dataset.map(tokenize, batched=True)
    tokenized.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "label"]
    )
    return tokenized


In [9]:
import transformers
print(transformers.__version__)


4.51.1


In [20]:
import wandb
from kaggle_secrets import UserSecretsClient

# Access the secret safely
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

# Log in to W&B
wandb.login(key=wandb_api_key)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sudhinkarki20 (sudhinkarki20-tribhuvan-university-institute-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [36]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import torch
import random
import numpy as np
import evaluate

accuracy_metric = evaluate.load("accuracy")

def set_seed(seed=42):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

set_seed(42)

dataset = load_dataset("glue", "sst2")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess(example):
    return tokenizer(example["sentence"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(preprocess, batched=True)

train_dataset = tokenized["train"].shuffle(seed=42).select(range(2000))
val_dataset = tokenized["validation"].select(range(500))

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Applied LoRA using PEFT
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="output",
    eval_strategy="epoch", 
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model("output/best_model")
tokenizer.save_pretrained("output/best_model")


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_31/3190166208.py:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700


No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.668900,0.678758,0.530000
2,0.591400,0.593848,0.654000
3,0.537600,0.514501,0.770000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


('output/best_model/tokenizer_config.json',
 'output/best_model/special_tokens_map.json',
 'output/best_model/vocab.txt',
 'output/best_model/added_tokens.json',
 'output/best_model/tokenizer.json')

In [24]:
def print_metrics(eval_results):
    print("\nEvaluation Results:")
    for key, value in eval_results.items():
        print(f"{key}: {value:.4f}")

In [26]:
print_metrics({'train_runtime': 88.6063, 'train_samples_per_second': 67.715, 'train_steps_per_second': 4.232, 'total_flos': 396032624640000.0, 'train_loss': 0.6782548421223958, 'epoch': 3.0})


Evaluation Results:
train_runtime: 88.6063
train_samples_per_second: 67.7150
train_steps_per_second: 4.2320
total_flos: 396032624640000.0000
train_loss: 0.6783
epoch: 3.0000


# Efficient Fine-tuning with PyTorch and PEFT (LoRA)

This project demonstrates how to:
- Fine-tune a transformer model (BERT) efficiently using LoRA (Low-Rank Adaptation)
- Use HuggingFace ecosystem (Transformers + Datasets + Trainer)
- Structure training with reusability, testing, and deployment in mind

## Frameworks Used
- PyTorch
- HuggingFace Transformers, Datasets
- PEFT (Parameter-Efficient Fine-Tuning)
- Accelerate for hardware abstraction

## Results
We fine-tuned bert-base-uncased on a 2k subset of the SST2 dataset using Low-Rank Adaptation (LoRA) with PEFT. After 3 epochs, the model reached a validation accuracy of 77.0%, improving from 53.0% in epoch 1 — showing effective training with minimal compute.